In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES']='6'

In [2]:
from transformers import AutoTokenizer
import re
import numpy as np
from random import Random
import torch
import pandas as pd
import spacy
import random
from datasets import load_dataset
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    TrainingArguments,
    Trainer,
    set_seed)
import numpy as np
import datasets
from collections import defaultdict
from datasets import load_metric

2023-02-19 21:50:14.807003: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-19 21:50:14.940255: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-19 21:50:14.940283: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-19 21:50:14.971344: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-19 21:50:15.785053: W tensorflow/stream_executor/platform/de

In [3]:
# !pip install seqeval

In [4]:
per_device_train_batch_size = 16
per_device_eval_batch_size = 32
num_train_epochs = 5
weight_decay = 0.1
warmup_ratio = 0.1
learning_rate = 5e-5
load_best_model_at_end = True
output_dir = "/mounts/work/akoksal/earthquake_ner_models/"
data_path = "annotated_address_dataset_07022023_766train_192test/"
cache_dir = "/mounts/work/akoksal/hf_cache"
saved_models_path = "/mounts/work/akoksal/earthquake_ner_models/"
device = "cuda"
seed = 42
model_names = ["dbmdz/bert-base-turkish-cased",
              "dbmdz/electra-base-turkish-mc4-cased-discriminator",
              "dbmdz/bert-base-turkish-128k-cased",
              "dbmdz/convbert-base-turkish-cased",
              "bert-base-multilingual-cased",
              "xlm-roberta-base"]
model_name = model_names[0]

In [5]:
model_name

'dbmdz/bert-base-turkish-cased'

In [6]:
set_seed(seed)

In [7]:
id2label = {
    0: "O",
    1: "B-no",
    2: "I-no",
    3: "B-mahalle",
    4: "I-mahalle",
    5: "B-ad-soyad",
    6: "I-ad-soyad",
    7: "B-kat",
    8: "I-kat",
    9: "B-ilce",
    10: "I-ilce",
    11: "B-sokak",
    12: "I-sokak",
    13: "B-dis kapi no",
    14: "I-dis kapi no",
    15: "B-Apartman/Site",
    16: "I-Apartman/Site",
    17: "B-ic kapi no",
    18: "I-ic kapi no",
    19: "B-il",
    20: "I-il"
}

label2id = {label: idx for idx, label in id2label.items()}
label_names = list(label2id.keys())

In [8]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name,
                                                        num_labels=len(label_names),
                                                        id2label=id2label,
                                                        cache_dir=cache_dir).to(device)

Some weights of the model checkpoint at dbmdz/bert-base-turkish-cased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initiali

In [9]:
dataset = datasets.load_from_disk(data_path)
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_dataset = dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [10]:
data_collator = DataCollatorForTokenClassification(tokenizer)

In [11]:
metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_names[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    flattened_results = {
        "overall_precision": results["overall_precision"],
        "overall_recall": results["overall_recall"],
        "overall_f1": results["overall_f1"],
        "overall_accuracy": results["overall_accuracy"],
    }
    for k in results.keys():
        if(k not in flattened_results.keys()):
            flattened_results[k+"_f1"]=results[k]["f1"]
            flattened_results[k+"_recall"]=results[k]["recall"]
            flattened_results[k+"_precision"]=results[k]["precision"]
            flattened_results[k+"_support"]=results[k]["number"]

    return flattened_results

/tmp/ipykernel_3903590/885599324.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [12]:
training_args = TrainingArguments(
    output_dir=saved_models_path,
    evaluation_strategy="epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    num_train_epochs=num_train_epochs,
    warmup_ratio=warmup_ratio,
    weight_decay=weight_decay,
    run_name = "turkish_ner",
    save_strategy='epoch',
    logging_strategy="epoch",
    save_total_limit=3,
    load_best_model_at_end=load_best_model_at_end
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [13]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, tokens, ner_tags. If id, tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
/mounts/work/akoksal/anaconda3/envs/lmbias/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 766
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 240
  Number of trainable parameters = 110042901
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, 

Epoch,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy,Apartman/site F1,Apartman/site Recall,Apartman/site Precision,Apartman/site Support,Ad-soyad F1,Ad-soyad Recall,Ad-soyad Precision,Ad-soyad Support,Dis kapi no F1,Dis kapi no Recall,Dis kapi no Precision,Dis kapi no Support,Ic kapi no F1,Ic kapi no Recall,Ic kapi no Precision,Ic kapi no Support,Il F1,Il Recall,Il Precision,Il Support,Ilce F1,Ilce Recall,Ilce Precision,Ilce Support,Kat F1,Kat Recall,Kat Precision,Kat Support,Mahalle F1,Mahalle Recall,Mahalle Precision,Mahalle Support,No F1,No Recall,No Precision,No Support,Sokak F1,Sokak Recall,Sokak Precision,Sokak Support
1,1.187100,0.342450,0.530374,0.663743,0.589610,0.903123,0.481283,0.600000,0.401786,75,0.765799,0.844262,0.700680,122,0.000000,0.000000,0.000000,9,0.000000,0.000000,0.000000,10,0.753247,0.820755,0.696000,106,0.573248,0.535714,0.616438,84,0.000000,0.000000,0.000000,13,0.611765,0.696429,0.545455,112,0.335570,0.595238,0.233645,42,0.546154,0.639640,0.476510,111
2,0.245700,0.285237,0.637500,0.745614,0.687332,0.915457,0.629834,0.760000,0.537736,75,0.787645,0.836066,0.744526,122,0.000000,0.000000,0.000000,9,0.000000,0.000000,0.000000,10,0.862222,0.915094,0.815126,106,0.819672,0.892857,0.757576,84,0.315789,0.461538,0.240000,13,0.630252,0.669643,0.595238,112,0.450980,0.547619,0.383333,42,0.638298,0.675676,0.604839,111
3,0.172800,0.257390,0.661125,0.755848,0.705321,0.924209,0.643678,0.746667,0.565657,75,0.792157,0.827869,0.759398,122,0.000000,0.000000,0.000000,9,0.266667,0.200000,0.400000,10,0.924528,0.924528,0.924528,106,0.822222,0.880952,0.770833,84,0.470588,0.615385,0.380952,13,0.629787,0.660714,0.601626,112,0.376068,0.523810,0.293333,42,0.697872,0.738739,0.661290,111
4,0.136400,0.252394,0.675497,0.745614,0.708826,0.926795,0.654088,0.693333,0.619048,75,0.809339,0.852459,0.770370,122,0.000000,0.000000,0.000000,9,0.181818,0.200000,0.166667,10,0.919431,0.915094,0.923810,106,0.828729,0.892857,0.773196,84,0.424242,0.538462,0.350000,13,0.666667,0.687500,0.647059,112,0.457143,0.571429,0.380952,42,0.623377,0.648649,0.600000,111
5,0.110900,0.264283,0.687166,0.751462,0.717877,0.927392,0.653846,0.680000,0.629630,75,0.814229,0.844262,0.786260,122,0.000000,0.000000,0.000000,9,0.272727,0.300000,0.250000,10,0.914286,0.905660,0.923077,106,0.837989,0.892857,0.789474,84,0.457143,0.615385,0.363636,13,0.632479,0.660714,0.606557,112,0.475248,0.571429,0.406780,42,0.689655,0.720721,0.661157,111


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, tokens, ner_tags. If id, tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 192
  Batch size = 32
/mounts/work/akoksal/anaconda3/envs/lmbias/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to /mounts/work/akoksal/earthquake_ner_models/checkpoint-48
Configuration saved in /mounts/work/akoksal/earthquake_ner_models/checkpoint-48/config.json
Model weights saved in /mounts/work/akoksal/earthquake_ner_models/checkpoint-48/pytorch_model.bin
tokenizer config file saved in /mounts/work/

TrainOutput(global_step=240, training_loss=0.3705816388130188, metrics={'train_runtime': 45.7151, 'train_samples_per_second': 83.78, 'train_steps_per_second': 5.25, 'total_flos': 178119641824452.0, 'train_loss': 0.3705816388130188, 'epoch': 5.0})

In [14]:
results = trainer.evaluate(tokenized_dataset["test"])

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: id, tokens, ner_tags. If id, tokens, ner_tags are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 192
  Batch size = 32


/mounts/work/akoksal/anaconda3/envs/lmbias/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [15]:
structured_results = defaultdict(dict)
structured_results["overall"]["support"]=0
for x, y in results.items():
    if len(x.split("_"))==3:
        structured_results[x.split("_")[1]][x.split("_")[2]] = y
        if x.split("_")[2]=="support":
            structured_results["overall"]["support"]+=y
results_pd = pd.DataFrame(structured_results).T
results_pd.support = results_pd.support.astype(int)
results_pd.round(2)

,support,precision,recall,f1,accuracy
overall,684,0.68,0.75,0.71,0.93
Apartman/Site,75,0.62,0.69,0.65,NaN
ad-soyad,122,0.77,0.85,0.81,NaN
dis kapi no,9,0.00,0.00,0.00,NaN
ic kapi no,10,0.17,0.20,0.18,NaN
il,106,0.92,0.92,0.92,NaN
ilce,84,0.77,0.89,0.83,NaN
kat,13,0.35,0.54,0.42,NaN
mahalle,112,0.65,0.69,0.67,NaN
no,42,0.38,0.57,0.46,NaN


## Predictions

In [16]:
from transformers import pipeline
nlp = pipeline("ner", model=model.to(device), tokenizer=tokenizer, aggregation_strategy="first", device=0 if device=="cuda" else -1)

In [17]:
# Source: https://www.thepythoncode.com/article/named-entity-recognition-using-transformers-and-spacy
def get_entities_html(text, ner_result, title=None):
    """Visualize NER with the help of SpaCy"""
    ents = []
    for ent in ner_result:
        e = {}
        # add the start and end positions of the entity
        e["start"] = ent["start"]
        e["end"] = ent["end"]
        # add the score if you want in the label
        # e["label"] = f"{ent["entity"]}-{ent['score']:.2f}"
        e["label"] = ent["entity_group"]
        if ents and -1 <= ent["start"] - ents[-1]["end"] <= 1 and ents[-1]["label"] == e["label"]:
            # if the current entity is shared with previous entity
            # simply extend the entity end position instead of adding a new one
            ents[-1]["end"] = e["end"]
            continue
        ents.append(e)
      # construct data required for displacy.render() method
    render_data = [
    {
      "text": text,
      "ents": ents,
      "title": title,
    }
    ]
    spacy.displacy.render(render_data, style="ent", manual=True, jupyter=True)

In [18]:
sentence = """Lütfen yardım Akevler mahallesi Rüzgar sokak Tuncay apartmanı zemin kat Antakya akrabalarım göçük altında #hatay #Afad"""

get_entities_html(sentence, nlp(sentence))

In [19]:
sentence = " ".join(dataset["train"][433]["tokens"])
get_entities_html(sentence, nlp(sentence))